**[Day4]**  24.3.25.월

<small>6.1.4 (214p) ~ 7.5 (267p)<small>

6장. 신경망을 활용한 데이터 적합

6.1 인공뉴런

6.1.2 오차함수

6.1.3 우리에게 필요한건 활성함수

6.1.4 그밖의 활성함수


6.2 파이토치 


In [3]:
#-CH5 코드 재입력-
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)

t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape



n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices



train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u



6.2.1 forward 대신 __call__ 사용하기

6.2.2 다시 선형 모델로

In [8]:
import torch.nn as nn

linear_model = nn.Linear(1, 1) #nn.Linear : 가중치와 편향을 무작위로 초기화함
linear_model(val_t_un)

tensor([[-1.7723],
        [-3.3129]], grad_fn=<AddmmBackward0>)

In [9]:
linear_model.weight

Parameter containing:
tensor([[-0.7478]], requires_grad=True)

In [10]:
linear_model.bias

Parameter containing:
tensor([0.8975], requires_grad=True)

In [12]:
#입력값으로 모듈 호출
x = torch.ones(1)
linear_model(x)

tensor([0.1496], grad_fn=<ViewBackward0>)

<배치 입력 만들기>

In [13]:
x = torch.ones(10,1)
linear_model(x)

tensor([[0.1496],
        [0.1496],
        [0.1496],
        [0.1496],
        [0.1496],
        [0.1496],
        [0.1496],
        [0.1496],
        [0.1496],
        [0.1496]], grad_fn=<AddmmBackward0>)